This is the data preprocessing part, all existing data (under "data" folder) has been preprocessed, if there is no new data you can skip this part.

In [4]:
from xml.etree import ElementTree as ET
from tqdm import tqdm
import csv
import pandas as pd
import spacy
import re
sp = spacy.load('de_core_news_lg')

2022-08-11 07:03:05.324672: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-11 07:03:05.324723: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Raw Data

## ESCO (Skills)

Downloaded from https://esco.ec.europa.eu/en/use-esco/download

Version: ESCO dataset - v1.0.7

Content: classification

Language: de

File type: csv

We only used the "skills_de.csv" which contains all information of skills themselves

## DIN PAS 1045 (Courses)

Privoded by Fraunhofer FOKUS


### Extract Course Information from XML Raw as CSV

In [13]:
def import_course(input_file):
    tree = ET.parse(input_file)
    courses = tree.findall('.//COURSE')

    header = ['course_name', 'course_id', 'course_description']
    data_list=[]
    
    for course in courses:
        name = course.find('CS_NAME').text
        id = course.find('CS_ID').text
        description = course.find('CS_DESC_LONG').text
        description = description.replace('\\','')
        data = [name, id, description]
        data_list.append(data)

    with open('../data/all_courses.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        # write the header
        writer.writerow(header)
        # write the data
        writer.writerows(data_list)

"2022-05-10_FOKUS_AWT_CompetencyExtraction_WB_Brandenburg_re-encoded.xml": complete dataset
    
"2022-05-10_FOKUS_AWT_CompetencyExtraction_WB_Brandenburg_control_re-encoded.xml": evaluation dataset

Choose one of those

In [14]:
# import_course('../data/2022-05-10_FOKUS_AWT_CompetencyExtraction_WB_Brandenburg_re-encoded.xml')
import_course('../data/2022-05-10_FOKUS_AWT_CompetencyExtraction_WB_Brandenburg_control_re-encoded.xml')

# Dataset 

##  Skill Dataset

In [11]:
skills = pd.read_csv('../data/skills_de.csv')[['conceptUri','preferredLabel','description']]
skills.columns = ['concept_uri','preferred_label','description']
skills.to_csv('../data/all_skills.csv', index = False)
skills

,concept_uri,preferred_label,description
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,Musikpersonal verwalten,Zuweisen und Verwalten der Aufgaben des Person...
1,http://data.europa.eu/esco/skill/00064735-8fad...,Strafvollzugsverfahren beaufsichtigen,Überwachen des Betriebs einer Justizvollzugsan...
2,http://data.europa.eu/esco/skill/000709ed-2be5...,nicht unterdrückende Praktiken anwenden,"Ermitteln von Repressionen in Gesellschaften, ..."
3,http://data.europa.eu/esco/skill/0007bdc2-dd15...,Einhaltung von Vorschriften von Eisenbahnfahrz...,"Kontrollieren von Fahrzeugen, Komponenten und ..."
4,http://data.europa.eu/esco/skill/00090cc1-1f27...,verfügbare Dienste ermitteln,Ermitteln der verschiedenen verfügbaren Dienst...
...,...,...,...
13886,http://data.europa.eu/esco/skill/ffef5eb3-a15e...,berufliche Leistungsfähigkeit von Nutzern/Nutz...,"Wiederherstellen der kognitiven, sensomotorisc..."
13887,http://data.europa.eu/esco/skill/fff0b074-5a76...,Beleuchtung in Transportgeräten einbauen,Einbau von Beleuchtungselementen in Transportg...
13888,http://data.europa.eu/esco/skill/fff0e2cd-d0bd...,Verarbeitung natürlicher Sprache,"Technologien, die es IKT-Geräten ermöglichen, ..."
13889,http://data.europa.eu/esco/skill/fff5bc45-b506...,Bauarbeiten koordinieren,Koordinierung der Tätigkeiten mehrerer Bauarbe...


## DIN PAS 1045 Course Description Dataset

In [12]:
courses = pd.read_csv('../data/all_courses.csv')[['course_id','course_name','course_description']]
courses.to_csv('../data/all_courses.csv', index = False)
courses

,course_id,course_name,course_description
0,4A873264-7ADD-DE47-3039-1FDA692E8164,"""Schwierige"" Klienten? - Mit Patienten, Angehö...",- Analyse strapaziöser Gesprächsmuster\n- Schw...
1,99BEDCEB-4FF3-3F8F-88B7-B30E50638F01,Aktuelles Arbeitsrecht 2022,Kurzbeschreibung\nDas Arbeitsrecht unterliegt ...
2,97FD41FD-1A91-179C-3EA5-6A23E2F436D5,Ambulante Pflege - Rechtssicher Handeln und Ha...,- Grundlagen der straf- und zivilrechtlichen H...
3,A0CC573A-79D7-79A4-5B22-F675C4F06950,Aufgaben des gesetzlichen Betreuers - Zur Refo...,Kurzbeschreibung\nNoch im Jahr 2020 plant der ...
4,7C449EF7-12A8-82FC-0E5E-BD2885FDB93C,Basisqualifikation für ungelernte Pflegekräfte...,- Alten- und Krankenpflege\n . Körperpflege\n...
...,...,...,...
16847,A1E046BE-1C4B-B977-C9B9-88D8D2860107,5 Monate Weiterbildung: Organisation & Führung...,Die aktuell vorherrschende Situation auf dem A...
16848,A1DF8EEB-D317-70BA-508F-AFC732369860,Conversion und Usability Experte,Ziel der Maßnahme ist es den Teilnehmern eine ...
16849,A1DD9D98-2BA3-3A11-0A24-72FC4C79422A,Digital Transformation Management,Ziel der Maßnahme ist es den Teilnehmern eine ...
16850,A1DD60F7-9B6E-ECEE-F1DF-53FED2DCCC5C,E-Commerce Geschäftsmodelle,Ziel der Maßnahme ist es den Teilnehmern eine ...


# Data Preprocessing

In [13]:
def process_ER(doc):
    doc = sp(doc)
    doc_processed = ''
    for token in doc:
        for punctuation in ['/', "/-", "-"]:
            if punctuation in token.text:
                temp = token.text.replace(punctuation,'')
                if len(temp) != 0: token = sp(temp)[0]
        if not token.is_stop and not token.is_digit:
            #and re.match('^[äöüÄÖÜßA-Za-z0-9_-]*$', word.text):
            token = token.lemma_.lower()
            if token != "--": doc_processed += token.strip() + ' '
    return doc_processed.strip()

def process_NN(doc):
    doc = sp(doc)
    doc_processed = ''
    for token in doc:
        if not token.is_space:
            if token.is_punct and token.text != ',' : doc_processed += '. ' 
            else: doc_processed += token.text + ' '
    return doc_processed[:-1]

## ESCO Skills Dataset

### Skill Labels

In [14]:
skills['processed_label'] = skills['preferred_label'].apply(process_ER)

In [15]:
skills['processed_label'].to_csv("../data/labels_processed.csv")
skills['processed_label']

0                                  musikpersonal verwalten
1                      strafvollzugsverfahr beaufsichtigen
2                           unterdrückend praktik anwenden
3        einhaltung vorschrift eisenbahnfahrzeuge überp...
4                               verfügbar dienst ermitteln
                               ...                        
13886    beruflich leistungsfähigkeit nutzer nutzerinn ...
13887                  beleuchtung transportgerät einbauen
13888                       verarbeitung natürlich sprache
13889                               bauarbeit koordinieren
13890               absturzsicherung bordbretter anbringen
Name: processed_label, Length: 13891, dtype: object

### Skill Information
We used "preferredLabel" and "description" columns together as the feature.

In [16]:
skills['info'] = skills['preferred_label'] + '. ' + skills['description']

In [17]:
skills['info'].to_csv("../data/skills_info.csv")
skills['info']

0        Musikpersonal verwalten, Zuweisen und Verwalte...
1        Strafvollzugsverfahren beaufsichtigen, Überwac...
2        nicht unterdrückende Praktiken anwenden, Ermit...
3        Einhaltung von Vorschriften von Eisenbahnfahrz...
4        verfügbare Dienste ermitteln, Ermitteln der ve...
                               ...                        
13886    berufliche Leistungsfähigkeit von Nutzern/Nutz...
13887    Beleuchtung in Transportgeräten einbauen, Einb...
13888    Verarbeitung natürlicher Sprache, Technologien...
13889    Bauarbeiten koordinieren, Koordinierung der Tä...
13890    Absturzsicherungen und Bordbretter anbringen, ...
Name: info, Length: 13891, dtype: object

## DIN PAS 1045 Course Description Dataset

### Course Information
We used "course_name" and "couse_description" columns together as the feature.

In [18]:
courses['course_info'] = courses['course_name'] + '. ' + courses['course_description']

In [19]:
courses['course_info_ER'] = courses['course_info'].apply(process_ER)

In [20]:
courses['course_info_ER'].to_csv("../data/courses_info_ER.csv")
courses['course_info_ER']

0        schwierig klient patient angehörige kollege cl...
1        aktuelles arbeitsrecht 2022. kurzbeschreibung ...
2        ambulant pflege rechtssicher handeln haftungsr...
3        aufgabe gesetzlich betreuer reform betreuungsr...
4        basisqualifikation ungelernt pflegekräft zerti...
                               ...                        
16847    monat weiterbildung organisation & führung lea...
16848    conversion usability experte ziel maßnahme tei...
16849    digital transformation management ziel maßnahm...
16850    ecommerce geschäftsmodell ziel maßnahme teilne...
16851    experte digital content creation teilnehmer di...
Name: course_info_ER, Length: 16852, dtype: object

In [21]:
courses['course_info_NN'] = courses['course_info'].apply(process_NN)

In [22]:
courses['course_info_NN'].to_csv("../data/courses_info_NN.csv")
courses['course_info_NN']

0        . Schwierige . Klienten . . Mit Patienten , An...
1        Aktuelles Arbeitsrecht 2022. Kurzbeschreibung ...
2        Ambulante Pflege . Rechtssicher Handeln und Ha...
3        Aufgaben des gesetzlichen Betreuers . Zur Refo...
4        Basisqualifikation für ungelernte Pflegekräfte...
                               ...                        
16847    5 Monate Weiterbildung . Organisation . Führun...
16848    Conversion und Usability Experte . Ziel der Ma...
16849    Digital Transformation Management . Ziel der M...
16850    E-Commerce Geschäftsmodelle . Ziel der Maßnahm...
16851    Experte im Digital Content Creation . Die Teil...
Name: course_info_NN, Length: 16852, dtype: object